# OpenAI APIの前準備

In [ ]:
# パッケージのインストール
!pip install -U openai

In [ ]:
import os
from google.colab import userdata

# 環境変数の準備 (左端の鍵アイコンでOPENAI_API_KEYを設定)
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

In [ ]:
from openai import OpenAI

# クライアントの準備
client = OpenAI()

# 埋め込みの実行

In [ ]:
# テキストの準備
text = "これはテストです。"

In [ ]:
# テキストをベクトル表現に変換
response =client.embeddings.create(
    input=text,
    model="text-embedding-3-small"
)
print(response.data[0].embedding)

In [ ]:
# ベクトル表現の長さの確認
print(len(response.data[0].embedding))

# 近傍探索

In [ ]:
# Faissパッケージのインストール
!pip install faiss-cpu

In [ ]:
import numpy as np

# 入力テキストをベクトル表現に変換
in_text = "今日は雨振らなくてよかった"
response =client.embeddings.create(
    input=in_text,
    model="text-embedding-3-small"
)

# numpy配列に変換
in_embeds = [record.embedding for record in response.data]
in_embeds = np.array(in_embeds).astype("float32")

In [ ]:
# 対象テキストをベクトル表現に変換
target_texts = [
    "好きな食べ物は何ですか?",
    "どこにお住まいですか?",
    "朝の電車は混みますね",
    "今日は良いお天気ですね",
    "最近景気悪いですね"]
response =client.embeddings.create(
    input=target_texts,
    model="text-embedding-3-small"
)

# numpy配列に変換
target_embeds = [record.embedding for record in response.data]
target_embeds = np.array(target_embeds).astype("float32")

In [ ]:
import faiss

# Faissのインデックス生成
index = faiss.IndexFlatL2(1536)

In [ ]:
# 対象テキストをインデックスに追加
index.add(target_embeds)

In [ ]:
# 入力テキストで近傍探索の実行
distances, indices = index.search(in_embeds, 1)

# 確認
print("distance:", distances[0][0])
print("index:", indices[0][0])
print("target_text:", target_texts[indices[0][0]])